In [1]:
!pip install foolbox

In [2]:
import foolbox as fb
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfDeepFoolAttack

In [3]:
from __future__ import print_function
import torch.nn.functional as F
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import warnings
import torch.optim as optim
import torch.nn as nn
import sys
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from datetime import datetime
from numpy.linalg import norm
import torchvision.models as models

In [4]:
class CNN(nn.Module):
    """CNN."""

    def __init__(self):
        """CNN Builder."""
        super(CNN, self).__init__()

        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.1),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
        )

        self.fc_layer = nn.Sequential(
            nn.Linear(4096, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        """Perform forward."""

        # conv layers
        x = self.conv_layer(x)

        # flatten
        x = x.view(x.size(0), -1)

        # fc layer
        x = self.fc_layer(x)

        return x

In [5]:
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_cnn = CNN()
model_cnn.load_state_dict(torch.load("model_cnn_cifar10.pt", map_location=device))
model_cnn = model_cnn.to(device)
model_cnn.eval()

softmax = nn.Softmax(dim=1)

def enable_dropout(model):
    """ Function to enable the dropout layers during test-time """
    for m in model.modules():
        if m.__class__.__name__.startswith('Dropout'):
            m.train()

corrects = []
corrects_tuple_list = []

reformatted_GMT_timestamp = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
print(reformatted_GMT_timestamp)

2021-09-06 19:44:40


In [6]:
class Normalize(nn.Module):
    def __init__(self, mean, std):
        super(Normalize, self).__init__()
        self.mean = torch.Tensor(mean)
        self.std = torch.Tensor(std)
    def forward(self, x):
        return (x - self.mean.type_as(x)[None,:,None,None]) / self.std.type_as(x)[None,:,None,None]

batch_size = 64
norm = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

transform = transforms.ToTensor()

test_data = datasets.CIFAR10(root='/CIFARDATA', train=False, download=True, transform=transform)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

Files already downloaded and verified


In [7]:
warnings.filterwarnings("ignore")

torch.manual_seed(2)
np.random.seed(2)

eps = 2./255
alpha = 0.2 * eps
num_iter = 10

num_restart = 5

count_successfull_attack = 0
count_unsuccessfull_attack = 0

In [8]:
def fgsm(image,model,epsilon, label):

    image = image.detach()

    item_count = image.shape[0]

    delta = torch.zeros_like(image, requires_grad=True)
    delta.grad = None

    model.eval()
    o = model(norm((image + delta)))
    loss = nn.CrossEntropyLoss(reduce=False)(o, label)
    loss = loss.reshape(1, item_count)
    
    loss.backward(torch.ones_like(loss))

    delta.data = delta + epsilon * delta.grad.detach().sign()
    delta.grad.zero_()

    perturbed_image = image + delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image.detach()

In [9]:

def rectified_fgsm(image,model,epsilon):

    torch.manual_seed(2)
    np.random.seed(2)

    item_count = image.shape[0]

    image = image.detach()

    delta = torch.zeros_like(image, requires_grad=True)
    
    delta.grad = None

    model.eval()

    with torch.no_grad():
        o = model(norm(image))
        o = softmax(o)
    init_pred = o.data.max(1, keepdim=True)[1]

    lbls = torch.squeeze(init_pred,1)

    enable_dropout(model)

    dropout_predictions = torch.zeros([50,item_count,10])

    for i in range(50):

        enable_dropout(model)
        output = model(norm((image + delta)))
        output = softmax(output)

        dropout_predictions[i] = output

    variance = torch.var(dropout_predictions, dim=0)

    var = variance.mean(1,True)
    var = var.reshape(1,item_count)
    var = var.to(device)

    model.eval()
    o = model(norm((image + delta)))
    loss = nn.CrossEntropyLoss(reduce=False)(o, lbls)
    loss = loss.reshape(1, item_count)

    loss.backward(torch.ones_like(var))
    delta_loss = delta.grad.detach().sign()
    delta.grad.zero_()
    delta.grad = None

    var.backward(torch.ones_like(var))
    delta_unc = delta.grad.detach().sign()
    delta.grad.zero_()
    delta.grad = None

    zeros = torch.zeros_like(delta_loss)
    delta_unc = torch.where(delta_unc != delta_loss, zeros, delta_unc)
    delta.data = (delta + epsilon * delta_unc)

    # model.eval()
    perturbed_image = image + delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image.detach()
  

In [10]:
def bim(image,model,epsilon, label, alpha, num_iter):

    image = image.detach()

    item_count = image.shape[0]

    delta = torch.zeros_like(image, requires_grad=True)
    delta.grad = None

    for t in range(num_iter):

        model.eval()
        o = model(norm((image + delta)))

        loss = nn.CrossEntropyLoss(reduce=False)(o, label)
        loss = loss.reshape(1, item_count)

        loss.backward(torch.ones_like(loss))

        delta.data = (delta + alpha * delta.grad.detach().sign()).clamp(-epsilon, epsilon)
        delta.grad.zero_()


    perturbed_image = image + delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image.detach()

In [11]:

def rectified_bim(image,model,epsilon, num_iter, alpha):

    torch.manual_seed(2)
    np.random.seed(2)

    item_count = image.shape[0]

    image = image.detach()

    delta = torch.zeros_like(image, requires_grad=True)
    
    delta.grad = None

    model.eval()

    with torch.no_grad():
        o = model(norm(image))
        o = softmax(o)
    init_pred = o.data.max(1, keepdim=True)[1]

    lbls = torch.squeeze(init_pred,1)

    enable_dropout(model)

    for t in range(num_iter):

        dropout_predictions = torch.zeros([50,item_count,10])

        for i in range(50):

            enable_dropout(model)
            output = model(norm((image + delta)))
            output = softmax(output)

            dropout_predictions[i] = output


        variance = torch.var(dropout_predictions, dim=0)

        var = variance.mean(1,True)
        var = var.reshape(1,item_count)
        var = var.to(device)

        model.eval()
        o = model(norm((image + delta)))
        loss = nn.CrossEntropyLoss(reduce=False)(o, lbls)
        loss = loss.reshape(1, item_count)

        if t == 0:

            loss.backward(torch.ones_like(var))
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            var.backward(torch.ones_like(var))
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            zeros = torch.zeros_like(delta_loss)
            delta_unc = torch.where(delta_unc != delta_loss, zeros, delta_unc)
            delta.data = (delta + alpha * delta_unc).clamp(-epsilon, epsilon)

        else:

            model.eval()
            with torch.no_grad():
                o = model(norm((image + delta)))
                o = softmax(o)
            inter_pred = o.data.max(1, keepdim=True)[1]

##########################################################################################

            inds_notmatch = np.where(inter_pred.cpu() != init_pred.cpu())[0]

            temp = torch.ones_like(var)
            temp = temp.cpu().numpy()

            temp[0][inds_notmatch] = 0
            temp = torch.tensor(temp)
            temp = temp.to(device)

            var.backward(temp, retain_graph=True)
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            loss.backward(temp, retain_graph=True)
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            zeros = torch.zeros_like(delta_loss)
            delta_loss = torch.where(delta_unc != delta_loss, zeros, delta_loss)
            delta.data = (delta + alpha * delta_loss).clamp(-epsilon, epsilon)

##########################################################################################

            inds_match = np.where(inter_pred.cpu() == init_pred.cpu())[0]

            temp = torch.ones_like(var)
            temp = temp.cpu().numpy()

            temp[0][inds_match] = 0
            temp = torch.tensor(temp)
            temp = temp.to(device)

            var.backward(temp)
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            loss.backward(temp)
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            zeros = torch.zeros_like(delta_loss)
            delta_loss = torch.where(delta_unc == delta_loss, zeros, delta_loss)
            delta.data = (delta + alpha * delta_loss).clamp(-epsilon, epsilon)

##########################################################################################

    # model.eval()
    perturbed_image = image + delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image.detach()
  

In [12]:
def pgd(image,model,epsilon, label, alpha, num_iter, restarts):

    torch.manual_seed(2)
    np.random.seed(2)

    item_count = image.shape[0]
    image = image.detach()

    max_loss = torch.zeros(label.shape[0]).to(label.device)
    max_delta = torch.zeros_like(image)

    for i in range(restarts):

        delta = torch.rand_like(image, requires_grad=True)
        delta.data = delta.data * 2 * epsilon - epsilon

        for t in range(num_iter):

            model.eval()
            o = model(norm((image + delta)))

            loss = nn.CrossEntropyLoss(reduce=False)(o, label)
            loss = loss.reshape(1, item_count)

            loss.backward(torch.ones_like(loss))

            delta.data = (delta + alpha * delta.grad.detach().sign()).clamp(-epsilon, epsilon)
            delta.grad.zero_()
        
        all_loss = nn.CrossEntropyLoss(reduction='none')(model(norm((image + delta))),label)
        max_delta[all_loss >= max_loss] = delta.detach()[all_loss >= max_loss]
        max_loss = torch.max(max_loss, all_loss)

    perturbed_image = image + max_delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image.detach()

In [13]:

def rectified_pgd(image, model, epsilon, label, alpha, num_iter, restarts):

    torch.manual_seed(2)
    np.random.seed(2)

    item_count = image.shape[0]

    image = image.detach()

    max_loss = torch.zeros(label.shape[0]).to(label.device)
    max_delta = torch.zeros_like(image)

    for r in range(restarts):

        delta = torch.rand_like(image, requires_grad=True)
        delta.data = delta.data * 2 * epsilon - epsilon

        model.eval()
        with torch.no_grad():
            o = model(norm(image))
            o = softmax(o)
        init_pred = o.data.max(1, keepdim=True)[1]

        lbls = torch.squeeze(init_pred,1)

        enable_dropout(model)

        for t in range(num_iter):

            dropout_predictions = torch.zeros([50,item_count,10])

            for i in range(50):

                enable_dropout(model)
                output = model(norm((image + delta)))
                output = softmax(output)

                dropout_predictions[i] = output

            variance = torch.var(dropout_predictions, dim=0)

            var = variance.mean(1,True)
            var = var.reshape(1,item_count)
            var = var.to(device)

            model.eval()
            o = model(norm((image + delta)))
            loss = nn.CrossEntropyLoss(reduce=False)(o, lbls)
            loss = loss.reshape(1, item_count)

            if t == 0:

                loss.backward(torch.ones_like(var))
                delta_loss = delta.grad.detach().sign()
                delta.grad.zero_()
                delta.grad = None

                var.backward(torch.ones_like(var))
                delta_unc = delta.grad.detach().sign()
                delta.grad.zero_()
                delta.grad = None

                zeros = torch.zeros_like(delta_loss)
                delta_unc = torch.where(delta_unc != delta_loss, zeros, delta_unc)
                delta.data = (delta + alpha * delta_unc).clamp(-epsilon, epsilon)

            else:

                model.eval()
                with torch.no_grad():
                    o = model(norm((image + delta)))
                    o = softmax(o)
                inter_pred = o.data.max(1, keepdim=True)[1]

    ##########################################################################################

                inds_notmatch = np.where(inter_pred.cpu() != init_pred.cpu())[0]

                temp = torch.ones_like(var)
                temp = temp.cpu().numpy()

                temp[0][inds_notmatch] = 0
                temp = torch.tensor(temp)
                temp = temp.to(device)

                var.backward(temp, retain_graph=True)
                delta_unc = delta.grad.detach().sign()
                delta.grad.zero_()
                delta.grad = None

                loss.backward(temp, retain_graph=True)
                delta_loss = delta.grad.detach().sign()
                delta.grad.zero_()
                delta.grad = None

                zeros = torch.zeros_like(delta_loss)
                delta_loss = torch.where(delta_unc != delta_loss, zeros, delta_loss)
                delta.data = (delta + alpha * delta_loss).clamp(-epsilon, epsilon)


    ##########################################################################################

                inds_match = np.where(inter_pred.cpu() == init_pred.cpu())[0]

                temp = torch.ones_like(var)
                temp = temp.cpu().numpy()

                temp[0][inds_match] = 0
                temp = torch.tensor(temp)
                temp = temp.to(device)

                var.backward(temp)
                delta_unc = delta.grad.detach().sign()
                delta.grad.zero_()
                delta.grad = None

                loss.backward(temp)
                delta_loss = delta.grad.detach().sign()
                delta.grad.zero_()
                delta.grad = None

                zeros = torch.zeros_like(delta_loss)
                delta_loss = torch.where(delta_unc == delta_loss, zeros, delta_loss)
                delta.data = (delta + alpha * delta_loss).clamp(-epsilon, epsilon)

    ##########################################################################################

        all_loss = nn.CrossEntropyLoss(reduction='none')(model(norm((image + delta))),label)
        max_delta[all_loss >= max_loss] = delta.detach()[all_loss >= max_loss]
        max_loss = torch.max(max_loss, all_loss)

    # model.eval()
    perturbed_image = image + max_delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image.detach()
  

In [14]:

for i, (image, label) in enumerate(test_loader):

    image = image.to(device)
    label = label.to(device)

    model_cnn.eval()

    with torch.no_grad():
        o = model_cnn(norm(image))
        o = softmax(o)

    pred_original = o.data.max(1, keepdim=True)[1]
    pred_original = pred_original.view_as(label)
    inds_correct = np.where(pred_original.cpu() == label.cpu())[0]

    image = image[inds_correct]
    label = label[inds_correct]


    preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)

    #pert = fgsm(image, model_cnn, eps, label)
    
    #pert = rectified_fgsm(image, model_cnn, eps)

    #pert = bim(image, model_cnn, eps, label, alpha, num_iter)

    #pert = rectified_bim(image, model_cnn, eps, num_iter, alpha)

    #pert = pgd(image, model_cnn, eps, label, alpha, num_iter, num_restart)
    
    pert = rectified_pgd(image, model_cnn, eps, label, alpha, num_iter, num_restart)

    #attack = LinfDeepFoolAttack()
    #fmodel = PyTorchModel(model_cnn, bounds=(0, 1),preprocessing=preprocessing)
    #raw_advs, clipped_advs, success = attack(fmodel, image, label, epsilons=[eps])
    #pert = torch.tensor(clipped_advs[0])

    model_cnn.eval()
    with torch.no_grad():
        o = model_cnn(norm(pert))
        o = softmax(o)
    pred_pert = o.data.max(1, keepdim=True)[1]

    pred_pert = pred_pert.view_as(label)

    inds_correct_after_attack = np.where(pred_pert.cpu() == label.cpu())[0]
    inds_wrong_after_attack = np.where(pred_pert.cpu() != label.cpu())[0]

    inds_correct_after_attackk = inds_correct_after_attack.tolist()
    inds_wrong_after_attackk = inds_wrong_after_attack.tolist()

    count_unsuccessfull_attack += len(inds_correct_after_attackk)
    count_successfull_attack += len(inds_wrong_after_attackk)

    image = image[inds_wrong_after_attack]
    label = label[inds_wrong_after_attack]
    pert = pert[inds_wrong_after_attack]

    if i%30==0:
      print("Success ratio for the attack is : %{:.2f}".format(
    100 * count_successfull_attack / (count_successfull_attack + count_unsuccessfull_attack)))

    #if i == 0:
        #break

Success ratio for the attack is : %65.31
Success ratio for the attack is : %65.81
Success ratio for the attack is : %65.75
Success ratio for the attack is : %66.20
Success ratio for the attack is : %66.95
Success ratio for the attack is : %67.12


In [15]:
print("Number of successful attack is : ", count_successfull_attack)
print("Number of unsuccessful attack is : ", count_unsuccessfull_attack)

print("Success ratio for the attack is : %{:.2f}".format(
    100 * count_successfull_attack / (count_successfull_attack + count_unsuccessfull_attack)))

reformatted_GMT_timestamp = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
print(reformatted_GMT_timestamp)

Number of successful attack is :  5587
Number of unsuccessful attack is :  2734
Success ratio for the attack is : %67.14
2021-09-07 00:50:54
